# This notebook stores useful code blocks for ZND Remote Control

## Manual Control
Add buttons on touchscreen to enable manual measuring, memorizing sweeps, clearing them, or exporting them all at once.

In [19]:
from Instrument import Instrument
from datetime import datetime
import Types
from time import sleep

In [20]:
# connect to the instrument
resource_string = 'TCPIP::192.168.0.169::hislip0'
instrument = Instrument(resource_string)
instrument.greet()


Hello, I am: 'Rohde-Schwarz,ZND-2Port,1328517092100889,2.80'
RsInstrument driver version: 1.18.0.73
Visa manufacturer: Rohde & Schwarz GmbH
Instrument full name: ZND-2Port
Instrument installed options: K1,K6


In [21]:
def measure():
    instrument.sweep_initiate_all()

In [22]:
def memorize():
    mem_trace_name = f'MemoryTrace{next(mem_index_gen)}'
    instrument.trace_copy(mem_trace_name, 'MainTrace')
    instrument.trace_assign_to_window(1, next(trc_index_gen), mem_trace_name)

In [23]:
def export():
    save_path = r'C:\Users\Instrument\Desktop\Data\Manual_{}.dat'.format(str(datetime.now().strftime("%b-%d-%Y_%H-%M-%S")))
    instrument.trace_save_all(1, save_path,
                             formatted=True,
                             save_format=Types.SaveFormat.COMPLEX,
                             dec_separator=Types.DecimalSeparator.POINT,
                             field_separator=Types.FieldSeparator.SEMICOLON)

In [24]:
def clear():
    global mem_index_gen, trc_index_gen
    instrument.trace_delete_all()
    instrument.trace_create(1, 'MainTrace', 'S21')
    instrument.trace_assign_to_window(1, 1, 'MainTrace')
    instrument.trace_format(1, Types.TraceFormat.LINEAR)
    instrument.sweep_initiate_all()
    instrument.trace_scale_auto(1)
    mem_index_gen = index_sequence(1)
    trc_index_gen = index_sequence(2)

In [26]:
# set up the instrument
instrument.reset()
instrument.trace_format(1, Types.TraceFormat.SMITH)
instrument.trace_rename(1, 'MainTrace')
instrument.correction_load(1, 'SMA ideal.cal')
instrument.trace_set_points(1, 41)
instrument.sweep_set_mode(1, Types.SweepMode.SINGLE)
instrument.sweep_initiate_all()
instrument.trace_scale_auto(1)
# TODO: set trace points to 1000 *
# TODO: set timeouts
# TODO: set calibration data *
# TODO: set display mode 
# TODO: set trace format *
mem_index_gen = index_sequence(1)
trc_index_gen = index_sequence(2)

In [123]:
# define buttons for manual control
instrument.button_define(Types.ButtonNumber.ONE, "Measure", measure)
instrument.button_define(Types.ButtonNumber.TWO, "Memorize", memorize)
instrument.button_define(Types.ButtonNumber.THREE, "Export", export)
instrument.button_define(Types.ButtonNumber.FOUR, "Clear", clear)
instrument.button_start_listening()

## Autopilot
Configure instrument and sweep every **sleep_time** seconds, total of **export_length** times, measuring S21 parameter of the antenna and saving it in "Data" folder on Desktop. Output has complex format. Process can be paused, resumed or stopped using buttons on the touchscreen.

In [27]:
from Instrument import Instrument
from datetime import datetime
import Types
from time import sleep

In [28]:
# connect to the instrument
resource_string = 'TCPIP::192.168.0.169::hislip0'
instrument = Instrument(resource_string)
instrument.greet()


Hello, I am: 'Rohde-Schwarz,ZND-2Port,1328517092100889,2.80'
RsInstrument driver version: 1.18.0.73
Visa manufacturer: Rohde & Schwarz GmbH
Instrument full name: ZND-2Port
Instrument installed options: K1,K6


In [29]:
def index_sequence(start_index):
    index = start_index
    while True:
        yield index
        index += 1

In [30]:
paused = False
stopped = False

def pause():
    global paused
    print("pause")
    paused = True
    
def resume():
    global paused
    print("resume")
    paused = False
    
def stop():
    global stopped
    print("stop")
    stopped = True

In [31]:
# add pause and resume buttons
instrument.button_define(Types.ButtonNumber.ONE, "Pause", pause)
instrument.button_define(Types.ButtonNumber.TWO, "Resume", resume)
instrument.button_define(Types.ButtonNumber.THREE, "Stop", stop)
instrument.button_start_listening()

In [32]:
# set up the instrument
instrument.reset()
instrument.trace_format(1, Types.TraceFormat.SMITH)
instrument.trace_rename(1, 'MainTrace')
instrument.correction_load(1, 'SMA ideal.cal')
instrument.trace_set_points(1, 50)
instrument.sweep_set_mode(1, Types.SweepMode.SINGLE)
instrument.write("frequency:start 2000000000") # start frequency
instrument.write("frequency:stop 5000000000")
# instrument.display_set_mode(Types.DisplayMode.OFF) # comment this for debugging
instrument.sweep_initiate_all()
instrument.trace_scale_auto(1)
mem_index_gen = index_sequence(1)
trc_index_gen = index_sequence(2)

In [34]:
# write autopilot loop
export_length = 100
sleep_time = 0
pause()

while True:
    if stopped:
        break
    if paused:
        continue
    
    for _ in range(export_length - 1):
        instrument.sweep_initiate_all()
        mem_trace_name = f'MemoryTrace{next(mem_index_gen)}'
        instrument.trace_copy(mem_trace_name, 'MainTrace')
        instrument.trace_assign_to_window(1, next(trc_index_gen), mem_trace_name) # uncomment this for debugging
        sleep(sleep_time)
    instrument.sweep_initiate_all()
        
    save_path = r'C:\Users\Instrument\Desktop\Data\Auto_RSand_{}.dat'.format(str(datetime.now().strftime("%b-%d-%Y_%H-%M-%S")))
    instrument.trace_save_all(1, save_path,
                             formatted=True,
                             save_format=Types.SaveFormat.COMPLEX,
                             dec_separator=Types.DecimalSeparator.POINT,
                             field_separator=Types.FieldSeparator.SEMICOLON)
    
    instrument.trace_delete_all()
    instrument.trace_create(1, 'MainTrace', 'S21')
    instrument.trace_assign_to_window(1, 1, 'MainTrace') # uncomment this for debugging
    instrument.trace_format(1, Types.TraceFormat.SMITH)
    instrument.sweep_initiate_all()
    instrument.trace_scale_auto(1) # uncomment this for debugging
    mem_index_gen = index_sequence(1)
    trc_index_gen = index_sequence(2)
    pause()

pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume
pause
resume

KeyboardInterrupt: 

In [43]:
from Steppers import Steppers
from Instrument import Instrument
from datetime import datetime
import Types
from time import sleep
import threading
import random

class StepperThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.steppers = Steppers(port='COM3')
        self.steppers.set_ignore_bounds(True)

    def run(self):
        step = 5
        max_side = 150 
        side = 0
        isX = True
        ystep = 0
        xstep = 0
        while True: 
          xstep = ystep = 0
          for i in range(max_side//step):
            if not isX:
              if ystep%2 == 1:
                self.steppers.move_y(side)
                # ymove(side)
              else:
                self.steppers.move_y(-side)
                # ymove(-side)
              side=side+step
              ystep += 1
              isX = not isX
            else:
              if xstep%2 == 1:
                self.steppers.move_x(-side)
                # xmove(side)
              else:
                self.steppers.move_x(side)
                # xmove(-side)
              side=side+step
              isX = not isX
              xstep += 1
            
          xstep = ystep = 0
          for i in range(max_side//step):
            if not isX:
              if ystep%2 == 1:
                self.steppers.move_y(-side)
                # ymove(side)
              else:
                self.steppers.move_y(side)
                # ymove(-side)
              side=side-step
              ystep += 1
              isX = not isX
            else:
              if xstep%2 == 1:
                self.steppers.move_x(side)
                # xmove(side)
              else:
                self.steppers.move_x(-side)
                # xmove(-side)
              side=side-step
              isX = not isX
              xstep += 1

In [44]:
stepper_thread = StepperThread()
stepper_thread.start()

Serial connection:  Serial<id=0x1757aa9a5b0, open=True>(port='COM3', baudrate=2000000, bytesize=8, parity='N', stopbits=1, timeout=60, xonxoff=False, rtscts=False, dsrdtr=False)
Note: Any read command will stop reading if the line contains '~' symbol.
Note: All occurrences of '~' symbol will be erased.
Program started.
Available commands:
1. movex
2. movey



## Steppers

In [11]:
from Steppers import Steppers
from time import sleep

In [29]:
steppers = Steppers(port='COM3')

Serial connection:  Serial<id=0x26178fa0100, open=True>(port='COM3', baudrate=2000000, bytesize=8, parity='N', stopbits=1, timeout=60, xonxoff=False, rtscts=False, dsrdtr=False)
Note: Any read command will stop reading if the line contains '~' symbol.
Note: All occurrences of '~' symbol will be erased.
Program started.
Available commands:
1. movex
2. movey



In [31]:
steppers.set_ignore_bounds(True)

In [17]:
steppers.release()

In [33]:
steppers.attach()

In [41]:
steppers.move_x(-20)

In [35]:
steppers.get_position()

(10, 0)

In [ ]:
steppers.set_position(0, 0)

In [ ]:
steppers.set_home()

In [ ]:
steppers.go_home()

## B-Scan

In [70]:
from Instrument import Instrument
from Steppers import Steppers
from datetime import datetime
import Types
from time import sleep

In [71]:
def b_scan(start, end, steppers, instrument, step_size_mm=10, axis=0, repeat_count=1, sleep_time=0):    
    mem_index_gen = index_sequence(1)
    
    def measure(x, y, measure_count=1):
        for _ in range(measure_count):
            instrument.sweep_initiate_all()
            mem_trace_name = f'MemoryTrace{next(mem_index_gen)}'
            instrument.trace_copy(mem_trace_name, 'MainTrace')
            sleep(sleep_time)
        instrument.sweep_initiate_all()
        
    step_number = int((abs(end - start)) / step_size_mm)
    direction = 1 if (end - start) > 0 else -1
    
    # go to start position
    x = start if axis == 0 else steppers.get_position()[0]
    y = start if axis == 1 else steppers.get_position()[1]
    steppers.set_position(x, y)
    
    
    for i in range(step_number):
        x = direction * step_size_mm if axis == 0 else 0
        y = direction * step_size_mm if axis == 1 else 0
        measure(x, y, repeat_count)
        steppers.move(x, y)
    measure(x, y, repeat_count - 1)
     
    # go to end position
    x = end if axis == 0 else steppers.get_position()[0]
    y = end if axis == 1 else steppers.get_position()[1]
    steppers.set_position(x, y)  
    
    # save file
    save_path = r'C:\Users\Instrument\Desktop\Data\Scan_{}_Start_{}_End_{}_Axis_{}.dat'.format(str(datetime.now().strftime("%b-%d-%Y_%H-%M-%S")), start, end, axis)
    instrument.trace_save_all(1, save_path,
                             formatted=True,
                             save_format=Types.SaveFormat.COMPLEX,
                             dec_separator=Types.DecimalSeparator.POINT,
                             field_separator=Types.FieldSeparator.SEMICOLON)

In [72]:
def index_sequence(start_index):
    index = start_index
    while True:
        yield index
        index += 1

In [83]:
def setup_instrument(instrument):
    # set up the instrument
    instrument.reset()
    instrument.trace_delete_all()
    instrument.trace_create(1, 'MainTrace', 'S21')
    instrument.trace_format(1, Types.TraceFormat.SMITH)
    instrument.correction_load(1, 'SMA actual.cal') # or 'SMA ideal.cal'
    # chose this bandwith to have 5cm resolution
    instrument.write("frequency:start 2000000000") # start frequency
    instrument.write("frequency:stop 5000000000") # stop frequency 
    instrument.trace_set_points(1, 50)
    instrument.sweep_set_mode(1, Types.SweepMode.SINGLE)
#     instrument.display_set_mode(Types.DisplayMode.OFF) # comment this for debugging
    instrument.sweep_initiate_all()

In [84]:
def setup_steppers(steppers):
    steppers.release()
    steppers.set_ignore_bounds(False)
    steppers.set_home()
    steppers.attach()
    steppers.set_max(500, 800)

In [75]:
# steppers = Steppers(port='COM3')

In [85]:
resource_string = 'TCPIP::192.168.0.169::hislip0'
instrument = Instrument(resource_string)
instrument.greet()


Hello, I am: 'Rohde-Schwarz,ZND-2Port,1328517092100889,2.80'
RsInstrument driver version: 1.9.0.52
Visa manufacturer: Rohde & Schwarz GmbH
Instrument full name: ZND-2Port
Instrument installed options: K1,K6


In [77]:
# setup_steppers(steppers)

In [86]:
setup_instrument(instrument)

In [65]:
# def c_scan(x_min, x_max, y_min, y_max, steppers, instrument, step_size_mm=10, sleep_time=0):
#     step_number_y = int((y_max - y_min) / step_size_mm)
    
#     for i in range(step_number_y):
#         b_scan(x_min, x_max, steppers, instrument, step_size_mm, 0, sleep_time)
        
#         # swap x_min and x_max
#         temp = x_min
#         x_min = x_max
#         x_max = temp
        
#         steppers.move(0, step_size_mm)

In [82]:
b_scan(0, 800, steppers, instrument, step_size_mm=20, axis=1, repeat_count=1, sleep_time=0)

In [ ]:
# c_scan(0, 500, 0, 800, steppers, instrument, 50, 0)

In [79]:
steppers = Steppers(port='COM3')

Serial connection:  Serial<id=0x26178fce550, open=True>(port='COM3', baudrate=2000000, bytesize=8, parity='N', stopbits=1, timeout=60, xonxoff=False, rtscts=False, dsrdtr=False)
Note: Any read command will stop reading if the line contains '~' symbol.
Note: All occurrences of '~' symbol will be erased.
Program started.
Available commands:
1. movex
2. movey



In [87]:
steppers.release()

In [81]:
steppers.attach()